# Importing the model

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

from tensorflow.keras.models import load_model
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.util import img_as_float
from PIL import Image

IMG_DIR = "/content/gdrive/MyDrive/Colab Notebooks/DEEP/user_photo.jpg"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Load the model with trained parameters

In [ ]:
os.chdir("/content/gdrive/MyDrive/Colab Notebooks/DEEP")
model = load_model("Model_incept")

labels = {0: 'M4_10', 1: 'M4_16', 2: 'M6_10', 3: 'M6_12', 4: 'None'}

In [ ]:
img_size = (256, 256)

def predict():
  img = Image.open(IMG_DIR)
  img = img.resize(img_size)
  img = img_as_float(img)
  img = np.expand_dims(img, axis=0)
  prediction = model.predict(img).argmax(axis=1)[0]
  return labels[prediction]

# Creating the Telegram Bot


In [ ]:
from telegram.ext import Updater, CallbackContext, CommandHandler, MessageHandler, Filters
from telegram import Update

In [ ]:
# Replace "TOKEN" with your private token key provided by telegram bot
updater = Updater(token='TOKEN', use_context=True)
dispatcher = updater.dispatcher

In [ ]:
def start(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text="Hello! Please send me square images of your screw")

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

In [ ]:
# Function that is called when a photo is sent
# Latest photo is retrieved and sent to the loaded model for prediction
# Finally, the returned prediction is sent back to the user

def photo(update: Update, context: CallbackContext):
    #context.bot.send_message(chat_id=update.effective_chat.id, text="Photo recieved!")
    photo_file = update.message.photo[-1].get_file()
    photo_file.download(IMG_DIR)
    output = "Unknown"
    try:
      output = predict()
    except:
      output = "Unknown"
    context.bot.send_message(chat_id=update.effective_chat.id, text="Prediction: " + output)



photo_handler = MessageHandler(Filters.photo, photo)
dispatcher.add_handler(photo_handler)

In [ ]:
updater.start_polling()

In [ ]:
#updater.stop()